In [60]:
import pandas as pd
import numpy as np

from nb import edhec_risk_kit_206 as erk

inds=['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
ind_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":"2018"]
ind_mcap = erk.get_ind_market_caps(49, weights=True)["2013-01"]

rets = ind_rets[inds]
rho = rets.corr()
vols = rets.std()*np.sqrt(12)
sigma_prior = vols.dot(vols.T) * rho

In [61]:
sigma_prior

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.108718,0.057064,0.070764,0.062429,0.040410
Fin,0.057064,0.108718,0.084545,0.064723,0.037292
Whlsl,0.070764,0.084545,0.108718,0.080859,0.058843
Rtail,0.062429,0.064723,0.080859,0.108718,0.062519
Food,0.040410,0.037292,0.058843,0.062519,0.108718


In [62]:
weights =ind_mcap[inds].T / ind_mcap[inds].T.sum()
w_eq = pd.DataFrame([0.041663,0.175362,0.097411,0.546388,0.139176], index=inds, columns=['CapWeight'])
w_eq.sort_values('CapWeight')

,CapWeight
Hlth,0.041663
Whlsl,0.097411
Food,0.139176
Fin,0.175362
Rtail,0.546388


In [63]:
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

pi = round(100*implied_returns(delta=2.5, sigma=sigma_prior, w=w_eq).sort_values(), 2)
pi

Hlth     15.29
Food     15.81
Fin      17.56
Whlsl    20.18
Rtail    22.48
Name: Implied Returns, dtype: float64

In [64]:
q = pd.Series([.03])
p = pd.DataFrame([0.]*len(inds), index=inds).T
w_rtail =  w_eq.loc["Rtail"]/(w_eq.loc["Rtail"]+w_eq.loc["Whlsl"])
w_whlsl =  w_eq.loc["Whlsl"]/(w_eq.loc["Rtail"]+w_eq.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_rtail
p.iloc[0]['Whlsl'] = -w_whlsl
(p*100).round(2)

,Hlth,Fin,Whlsl,Rtail,Food
0,100.0,0.0,-15.13,-84.87,0.0


In [65]:
from numpy.linalg import inv

# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)


def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

In [66]:
delta = 2.5
tau = 0.05
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(2).sort_values()

Food     14.53
Fin      16.93
Hlth     17.96
Whlsl    19.33
Rtail    19.98
dtype: float64

In [67]:
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w

In [68]:
w_msr(bl_sigma, bl_mu).sort_values()

Whlsl    0.061820
Food     0.139176
Fin      0.175362
Hlth     0.276885
Rtail    0.346756
dtype: float64

In [69]:
q = pd.Series([.05])

In [70]:
delta = 2.5
tau = 0.05
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(2).sort_values()

Food     14.27
Fin      16.80
Hlth     18.51
Whlsl    19.16
Rtail    19.48
dtype: float64

In [71]:
w_msr(bl_sigma, bl_mu).sort_values()

Whlsl    0.054342
Food     0.139176
Fin      0.175362
Rtail    0.304808
Hlth     0.326312
dtype: float64